# Extensión de la API de Pandas

In [ ]:
!pip install pandas==1.5.3 upsetplot==0.6.1

In [ ]:
#Importando las librerias a usar para funciones
import itertools
import pandas as pd
import upsetplot

In [ ]:
#Este bloque solo nos dice que si existe el acceso missing, eliminalo, si no, continua
#Esto es para evitar el missing que tenemos
try:
    del pd.DataFrame.missing
except AttributeError:
    pass

In [ ]:
@pd.api.extensions.register_dataframe_accessor("missing")
class MissingMethods:
    def __init__(self, pandas_obj):
        self._obj = pandas_obj

    def number_missing(self) -> int:
      # Retorna la cantidad total de valores faltantes en el DataFrame
      return self._obj.isna().sum().sum()
      # Identifica los valores faltantes con isna() y utiliza sum() dos veces para calcular la cantidad total de valores faltantes en el DataFrame

    def number_complete(self) -> int:
      # Retorna la cantidad total de valores completos (no faltantes) en el DataFrame
      return self._obj.size - self._obj.missing.number_missing()
      # Calcula la cantidad total de elementos en el DataFrame y resta la cantidad de valores faltantes obtenida a través de number_missing() para obtener la cantidad de valores completos

    def missing_variable_summary(self) -> pd.DataFrame:
      # Genera un resumen de variables con valores faltantes en el DataFrame
      return self._obj.isnull().pipe(
          lambda df_1: (
              df_1.sum()  # Calcula la cantidad de valores faltantes por columna
              .reset_index(name="n_missing")  # Resetea el índice y nombra la columna con la cantidad de valores faltantes como 'n_missing'
              .rename(columns={"index": "variable"})  # Renombra la columna del índice como 'variable'
              .assign( #Agrega columnas adicionales al Dataframe resultante
                   n_cases=len(df_1),  # Calcula el número total de casos (filas) en el DataFrames
                   pct_missing=lambda df_2: df_2.n_missing / df_2.n_cases * 100  # Calcula el porcentaje de valores faltantes por variable
              )
        )
    )

    def missing_case_summary(self) -> pd.DataFrame:
      #Genera un resumen de casos (filas) con valores faltantes en el Dataframe
        return self._obj.assign( #Asigna nuevas columnas al Df
            case=lambda df: df.index, #Crea una columna "case" con los indices del Df
            n_missing=lambda df: df.apply( #Crea una columna "n_mising" que cuenta los valores faltantes por fila
                axis="columns", func=lambda row: row.isna().sum() #Usa appply para aplicar una funcion lambda que cuenta los valores faltantes por fila
            ),
            pct_missing=lambda df: df["n_missing"] / df.shape[1] * 100, #Calcula el porcentaje de valores faltantes por fila
        )[["case", "n_missing", "pct_missing"]] #Selecciona las columnas "case", "n.missing" y "pct_missing"

    def missing_variable_table(self) -> pd.DataFrame:
        #Genera una tabla resumen de las variables con valores faltantes en el DataFrame
        return (
            self._obj.missing.missing_variable_summary() #Obtiene el resumen de variables con valores faltantes
            .value_counts("n_missing") #Cuenta la frecuencia de valores faltantes en las variables
            .reset_index() #Reinicia el indice del DataFrame resultante
            .rename(columns={"n_missing": "n_missing_in_variable", 0: "n_variables"}) #Renombra las columnas
            .assign(
                pct_variables=lambda df: df.n_variables / df.n_variables.sum() * 100 #Calcula el porcentaje de variables con faltantes
            )
            .sort_values("pct_variables", ascending=False) #Ordena porcentaje de variables faltantes en orden descendente
        )

    def missing_case_table(self) -> pd.DataFrame():
      #Genera una tabla resumen de casos con valores faltantes en el DataFrame
        return (
            self._obj.missing.missing_case_summary() #Obtiene el resumen de casos con valores faltantes
            .value_counts("n_missing") #Cuenta la frecuencia de valores faltantes en los casos
            .reset_index() #Reinicia el indice del DataFrame resultante
            .rename(columns={"n_missing": "n_missing_in_case", 0: "n_cases"}) #Renombra las columnas
            .assign(pct_case=lambda df: df.n_cases / df.n_cases.sum() * 100) #Calcula el porcentaje de casos con faltantes
            .sort_values("pct_case", ascending=False) #Ordena porcentaje de casos con valores faltantes en orden descendente
        )

    def missing_variable_span(self, variable: str, span_every: int) -> pd.DataFrame:
      #Crea un resumen de valores  faltantes por intervalos (especificados por span_every)
        return (
            self._obj.assign(
                #Crea un contador para agrupar los datos en intervalos
                span_counter=lambda df: (
                    np.repeat(a=range(df.shape[0]), repeats=span_every)[: df.shape[0]]
                )
            )
            .groupby("span_counter") # Agrupa por el contador de intervalos
            .aggregate(
                # Calcula la cantidad de valores en el intervalo
                n_in_span=(variable, "size"),
                # Cuenta los valores faltantes en el intervalo
                n_missing=(variable, lambda s: s.isnull().sum()),
            )
            .assign(
                # Calcula la cantidad de valores completos en el intervalo
                n_complete=lambda df: df.n_in_span - df.n_missing,
                # Calcula el porcentaje de valores faltantes en el intervalo
                pct_missing=lambda df: df.n_missing / df.n_in_span * 100,
                # Calcula el porcentaje de valores completos en el intervalo
                pct_complete=lambda df: 100 - df.pct_missing,
            )
            .drop(columns=["n_in_span"]) # Elimina la columna de valores en el intervalo
            .reset_index() # Reinicia el índice del DataFrame resultante
        )

    def missing_variable_run(self, variable) -> pd.DataFrame:
        # Crea una lista de secuencias de valores faltantes y completos en la variable dada
        rle_list = self._obj[variable].pipe(
            lambda s: [[len(list(g)), k] for k, g in itertools.groupby(s.isnull())]
        )

        # Crea un DataFrame con las longitudes de las secuencias y las etiquetas "missing" y "complete"
        return pd.DataFrame(data=rle_list, columns=["run_length", "is_na"]).replace(
            {False: "complete", True: "missing"}
        )

    def sort_variables_by_missingness(self, ascending = False):
      # Retorna el DataFrame con las columnas ordenadas por la cantidad de valores faltantes
        return (
            self._obj # Hace referencia al DataFrame al que se aplica esta función
            .pipe(
                lambda df: ( # Filtra el DataFrame según el orden de las columnas de valores faltantes
                    df[df.isna().sum().sort_values(ascending = ascending).index]
                )
            )
        )

    def create_shadow_matrix( #Crea una matriz de sombra que representa valores faltantes en el DataFrame.
        self,
        true_string: str = "Missing", # Cadena de texto que representa valores faltantes. Por defecto, es "Missing".
        false_string: str = "Not Missing", #Cadena de texto que representa valores no faltantes. Por defecto, es "Not Missing".
        only_missing: bool = False, # Indica si se deben considerar solo los valores faltantes. Por defecto, es False.
        suffix: str = "_NA", #Sufijo para las columnas de la matriz de sombra. Por defecto, es "_NA".
    ) -> pd.DataFrame:
        return (
            self._obj
            .isna()  #Genera una matriz booleana donde True indica valores faltantes
            .pipe(lambda df: df[df.columns[df.any()]] if only_missing else df) # Filtra las columnas que contienen valores faltantes (si 'only_missing' es True)
            .replace({False: false_string, True: true_string}) #Reemplaza los booleanos por strings
            .add_suffix(suffix) #Agrega un sufijo a los nombres de las columnas
        ) # Matriz de sombra que representa los valores faltantes en el DataFrame.

    def bind_shadow_matrix(  #Une la matriz de sombra al DataFrame original.
        self,
        true_string: str = "Missing", # Cadena de texto que representa valores faltantes. Por defecto, es "Missing".
        false_string: str = "Not Missing", #Cadena de texto que representa valores no faltantes. Por defecto, es "Not Missing"
        only_missing: bool = False, #Booleano que indica si se deben considerar solo los valores faltantes. Por defecto, es False.
        suffix: str = "_NA", #Sufijo para las columnas de la matriz de sombra. Por defecto, es "_NA".
    ) -> pd.DataFrame:
        return pd.concat(
            objs=[
                self._obj, #DataFrame original
                self._obj.missing.create_shadow_matrix( # Matriz de sombra creada por create_shadow_matrix
                    true_string=true_string,
                    false_string=false_string,
                    only_missing=only_missing,
                    suffix=suffix,
                ),
            ],
            axis="columns" # Concatenación a lo largo de las columnas
        )

    def missing_scan_count(self, search) -> pd.DataFrame: #    Cuenta la cantidad de valores faltantes que coinciden con una búsqueda especificada en el DataFrame.
        return (
            self._obj.apply(axis="rows", func=lambda column: column.isin(search)) # Comprueba si los valores de cada columna coinciden con la búsqueda
            .sum() # Suma los valores True resultantes de la búsqueda para cada columna
            .reset_index() # Reinicia el índice del DataFrame resultante
            .rename(columns={"index": "variable", 0: "n"}) #Renombra las columnas para claridad
            .assign(original_type=self._obj.dtypes.reset_index()[0]) #Añade una columna 'original_type' con los tipos de datos originales del DataFrame
        )

    # funcion que añade valores aleatorios a las variables con valores faltantes para visualizarlos en un eje
    def column_fill_with_dummies(
        self,
        column: pd.Series,
        proportion_below: float=0.10, #Proporcion de los datos en la grafica
        jitter: float=0.075,  # evita el asolapamiento de los puntos en la grafica
        seed: int=42, #semilla para la aleatoriedad
        ) -> pd.Series: # la funcion retorna una serie

        #Copiar las columnas del dataframe
        column = column.copy(deep=True)

        #Extraer los valores de las variables
        missing_mask = column.isna() # matriz de booleanos para ver si hay valores faltantes o no
        number_missing_values = missing_mask.sum() #conteo de valores faltantes
        column_range = column.max() - column.min() #rango de las variables

        # shift data, es decir de que pasemos todos nuestros datos por debajo de su valor original con la porporcion definida
        column_shift = column.min() - column.min() * proportion_below #Esto es para que los valores esten en una esquina y no molesten

        # crear un poco de ruido alrededor de los puntos,por si existe asolapamiento se puedan identificar
        np.random.seed(seed)
        column_jitter = (np.random.rand(number_missing_values) - 2) * column_range * jitter #Columna con los valores ya movidos del lugar

        #Guardar los nuevos datos aleatorios
        column[missing_mask] = column_shift + column_jitter

        return column

    # Plotting functions ---

    def missing_variable_plot(self): # Crea un gráfico de líneas horizontales para visualizar la cantidad de valores faltantes por variable en el DataFrame.

        # Obtiene un resumen de las variables y la cantidad de valores faltantes, ordenado por cantidad de valores faltantes
        df = self._obj.missing.missing_variable_summary().sort_values("n_missing")

        # Define un rango para el gráfico basado en el número de variables
        plot_range = range(1, len(df.index) + 1)

        # Dibuja líneas horizontales para representar la cantidad de valores faltantes por variable
        plt.hlines(y=plot_range, xmin=0, xmax=df.n_missing, color="black")

        #Dibuja puntos para mostrar la cantidad de valores faltantes por variable
        plt.plot(df.n_missing, plot_range, "o", color="black")

        #Etiqueta los ejes y ajusta los nombres de las variables en el eje y
        plt.yticks(plot_range, df.variable)
        plt.grid(axis="y")

        plt.xlabel("Number missing") #Etiqueta del eje x
        plt.ylabel("Variable") # Etiqueta del eje y

    def missing_case_plot(self): # Crea un gráfico de distribución para mostrar la cantidad de casos con valores faltantes.

        # Obtiene un resumen de los casos y la cantidad de valores faltantes por caso
        df = self._obj.missing.missing_case_summary()

        # Crea un gráfico de distribución con seaborn para visualizar la cantidad de valores faltantes por caso
        sns.displot(data=df, x="n_missing", binwidth=1, color="black")

        # Añade una cuadrícula al eje x para mejorar la visualización
        plt.grid(axis="x")

        # Etiqueta los ejes x e y
        plt.xlabel("Number of missings in case") # Etiqueta del eje x
        plt.ylabel("Number of cases") # Etiqueta del eje y

    def missing_variable_span_plot( # Crea un gráfico de barras que muestra el porcentaje de valores faltantes y presentes en un intervalo dado.
        self, variable: str, span_every: int, rot: int = 0, figsize=None
    ):
        # Calcula el porcentaje de valores faltantes y presentes en un intervalo específico

        (
            self._obj.missing.missing_variable_span(
                variable=variable, span_every=span_every
            ).plot.bar(
                x="span_counter",
                y=["pct_missing", "pct_complete"],
                stacked=True,
                width=1,
                color=["black", "lightgray"],
                rot=rot,
                figsize=figsize,
            )
        )

        #Etiqueta los ejes x e y, agrega leyenda y título
        plt.xlabel("Span number") # Etiqueta del eje x
        plt.ylabel("Percentage missing") # Etiqueta del eje y
        plt.legend(["Missing", "Present"]) #Leyenda del gráfico
        plt.title(
            f"Percentage of missing values\nOver a repeating span of { span_every } ",
            loc="left",
        ) #Título del gráfico
        plt.grid(False) # Elimina la cuadrícula del gráfico
        plt.margins(0) # Establece los márgenes a 0
        plt.tight_layout(pad=0) # Ajusta el diseño para evitar solapamientos

    def missing_upsetplot(self, variables: list[str] = None, **kwargs): #Crea un gráfico UpSet que muestra la intersección de valores faltantes entre múltiples variables.

        # Si no se especifican variables, se utilizan todas las columnas del DataFrame
        if variables is None:
            variables = self._obj.columns.tolist()

        # Calcula la intersección de valores faltantes entre las variables y crea el gráfico UpSet
        return (
            self._obj.isna() # Identifica los valores faltantes en el DataFrame
            .value_counts(variables) # Cuenta la combinación de valores faltantes para las variables
            .pipe(lambda df: upsetplot.plot(df, **kwargs)) # Crea el gráfico UpSet con los datos obtenidos
        )

    def scatter_missing_plot_2_variables(self, column1, column2):
      (
          self._obj
          .select_dtypes(exclude='category') #Solo tendra en cuentas las variables numericas, exlucye las categoricas
          .pipe(
              lambda df: df[df.columns[df.isna().any()]] #seleccionar solo columnas que tengan valores faltantes
              )
          .missing.bind_shadow_matrix(true_string=True, false_string=False) #Modifcara los valores missing y not missing por True y False
          .apply(lambda column : column if '_NA' in column.name else column_fill_with_dummies(column, proportion_below=0.05, jitter = 0.075 )) #lambda que tomara una columna y en caso que esta tenga el subijo _NA regresa el nombre y aquellas que son reales, numeros, las rellenemos con dummies para no tener valores nulos
          .assign(nullity = lambda df: df[column1 + '_NA'] | df[column2 + '_NA']) #se crea una columna que tendra las 2 variables que queremos graficar para podder identificar al momento de graficar
          .pipe( #Aca simplmente graficamos como se sabe
                lambda df : (
                    sns.scatterplot(
                    data = df,
                    x=column1,
                    y=column2,
                    hue='nullity'
                    )
              )
       )
  )

    def scatter_imputation_plot(self, x, y, imputation_suffix="_imp", show_marginal=False, **kwargs): # Crea un gráfico de dispersión que muestra valores imputados de dos variables.

      # Crea los nombres de las variables imputadas
      x_imputed = f"{ x }{ imputation_suffix }"
      y_imputed = f"{ y }{ imputation_suffix }"

      #  Decide qué tipo de gráfico utilizar: scatterplot o jointplot
      plot_func = sns.scatterplot if not show_marginal else sns.jointplot

      # Filtra las columnas relevantes y asigna una etiqueta para indicar si hay valores imputados
      return (
          self._obj[[x, y, x_imputed, y_imputed]]
          .assign(is_imputed=lambda df: df[x_imputed] | df[y_imputed]) #  Columna para identificar valores imputados
          .pipe(lambda df: (plot_func(data=df, x=x, y=y, hue="is_imputed", **kwargs))) #  Crea el gráfico
          )

    def missing_mosaic_plot(self,target_var: str, x_categorical_var: str,y_categorical_var: str, ax=None):
      return (
          self._obj
          .assign( #Crea una columna para la variable objetivo (indicando valores faltantes)
              **{target_var: lambda df: df.weight.isna().replace([True, False], ["NA", "!NA"])}
          )
          .groupby(
              [x_categorical_var, y_categorical_var, target_var], # Primero debemos cuantificar cuantas apariciones existen por cada combinacion de las 3 columns por eso usamos groupby y una lista las variables que queramos usar
              dropna=False, #Al haber faltantes en general, vamos a dejarlo asi para que no los elimine para que en la grafica se vea
              as_index=True, # Esto es para que lo regrese como indice
          )
          .size()
          .pipe(
              lambda df: mosaic( # Esta funcion viene de statsmodels
                  data=df, # Es el df que usaremos
                  properties=lambda key: {"color": "r" if "NA" in key else "gray"}, #Esto es que si es faltante pinta rojo y si no gris
                  ax=ax,
                  horizontal=True, # Cambiamos la orentacion del grafico
                  axes_label=True, # Esto es para dibuje los axes de los labels
                  title="", # Para poner un titulo
                  labelizer=lambda key: "", #Esto pone el texto en cada una de las cajas del grafico porque se amontonan
              )
          )
      )



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=694a3d08-7f18-421d-9e2f-c2820a79680e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>